In [1]:
from sklearn.model_selection import StratifiedKFold,KFold
from keras.preprocessing.text import one_hot
#from keras.preprocessing.text import pad_sequences
from keras.preprocessing.text import text_to_word_sequence
from keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.callbacks import History
from numpy import zeros
from keras.layers.embeddings import Embedding
import pandas as pd
import numpy as np
from prf1 import precision, recall, f1_score
from keras.utils.vis_utils import plot_model
#import pydot
#import graphviz
from keras import backend as K
from keras.preprocessing import sequence
from keras.layers import LSTM
import numpy
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
#from confusionMetrics import plot_confusion_matrix
import matplotlib.pyplot as plt
from sklearn import metrics
from prf1 import precision, recall, f1_score
from keras.utils import np_utils
from imblearn.over_sampling import SMOTE

Using TensorFlow backend.
C:\Users\Lenovo\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
#Data extraction and labelling
df=pd.read_excel('Tata_steel.xlsx')
#print(df.info())
df1=df['Field2']
label =df['LABEL']
tk=Tokenizer()
tk.fit_on_texts(df1)
index=tk.word_index
#print(index)
#x = tk.texts_to_sequences(df1)
#seed = 7
#numpy.random.seed(seed)
##encoded_doc = tk.texts_to_matrix(df, mode='count')
##print (encoded_doc)
##max_length=15
##padded_docs = sequence.pad_sequences(encoded_doc, maxlen=max_length, padding='pre')
##print (padded_docs)
vocab_size = len(index)
encoded_docs=[one_hot(d,vocab_size) for d in df1] 
#print(encoded_docs)
##############################Padding###############
max_length=50
padded_docs = sequence.pad_sequences(encoded_docs, maxlen=max_length, padding='post')
#print(padded_docs)
#####################Encoded label######
x_train1, x_val1, y_train1, y_val1=train_test_split(padded_docs,label, test_size=0.20,random_state=42)
#print(x_train1.shape)

In [8]:
#print(x_val1.shape)
#print(y_train1.shape)
#print(y_val1.shape)
sm = SMOTE(kind='regular')
x_train,Y_train = sm.fit_sample(x_train1,y_train1)
Y_train = np_utils.to_categorical(Y_train)
x_val,y_val = sm.fit_sample(x_val1,y_val1)
y_val = np_utils.to_categorical(y_val)

#print('after smote',x_train.shape)
#print(Y_train.shape)
#print(Y_train)

In [9]:
embeddings_index = {}
f = open('glove.6B.100d.txt',encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
 
print('Total %s word vectors in Glove 6B 100d.' % len(embeddings_index))
 
# In[10]:
embedding_matrix =zeros((vocab_size+ 1, 100))
for word, i in tk.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector   
print ("Embedding Matrix")

Total 400000 word vectors in Glove 6B 100d.
Embedding Matrix


In [11]:

 
 
 
#############embedding layers#########################
 
history=History()
predictions_train=[]
predictions_test=[]
filter_sizes = [3,4,5]
fold_training=numpy.zeros(shape=50)
fold_test=numpy.zeros(shape=50)
##embedding_vecor_length =32
model = Sequential()
model.add(Embedding(vocab_size+1, 100,
                            weights=[embedding_matrix],
                            input_length=max_length,
                            trainable=False))
##model.add(Embedding(vocab_size,8, input_length=embedding_vecor_length))
#kfold = KFold(n_splits=20, shuffle=True, random_state=7)
#cvscores = []
#ith=1
#for train,test in kfold.split(padded_docs,labels):
    #print('Fold=',ith)
    #ith=ith+1
    #print(train,test)
    #X_train, X_test = padded_docs[train], padded_docs[test]
    #Y_train, Y_test = labels[train], labels[test]
model.add(LSTM(200,return_sequences=True))
#model.add(LSTM(100,return_sequences=True))
#model.add(LSTM(200,return_sequences=True))
#model.add(LSTM(200,return_sequences=True,recurrent_dropout=0.2))
model.add(LSTM(100))
model.add(Dense(4, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer='RMSprop',
              metrics=['acc', precision, recall, f1_score])
#model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', precision,recall,f1_score])
print(model.summary())
history=model.fit(x_train, Y_train,validation_data=(x_val,y_val),epochs=75, batch_size=16,verbose=2, callbacks=[history])
predictions = model.predict(x_val)
b=np.zeros_like(predictions)
b[np.arange(len(predictions)), predictions.argmax(1)]=1
print( metrics.classification_report(y_val,b))



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 50, 100)           125600    
_________________________________________________________________
lstm_3 (LSTM)                (None, 50, 200)           240800    
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               120400    
_________________________________________________________________
dense_2 (Dense)              (None, 4)                 404       
Total params: 487,204
Trainable params: 361,604
Non-trainable params: 125,600
_________________________________________________________________
None
Train on 480 samples, validate on 136 samples
Epoch 1/75
 - 11s - loss: 1.3563 - acc: 0.2729 - precision: nan - recall: 0.0354 - f1_score: nan - val_loss: 1.5102 - val_acc: 0.1912 - val_precision: nan - val_recall: 0.0368 - val_f1_score: nan
Epoch 2/75
 - 9

Epoch 37/75
 - 8s - loss: 0.3343 - acc: 0.8917 - precision: 0.8992 - recall: 0.8833 - f1_score: 0.8909 - val_loss: 2.4099 - val_acc: 0.4118 - val_precision: 0.4221 - val_recall: 0.4118 - val_f1_score: 0.4167
Epoch 38/75
 - 7s - loss: 0.3269 - acc: 0.8896 - precision: 0.8975 - recall: 0.8771 - f1_score: 0.8869 - val_loss: 1.9203 - val_acc: 0.4338 - val_precision: 0.4503 - val_recall: 0.4191 - val_f1_score: 0.4337
Epoch 39/75
 - 7s - loss: 0.3230 - acc: 0.8896 - precision: 0.8959 - recall: 0.8750 - f1_score: 0.8850 - val_loss: 1.7847 - val_acc: 0.4118 - val_precision: 0.4378 - val_recall: 0.3897 - val_f1_score: 0.4110
Epoch 40/75
 - 9s - loss: 0.3070 - acc: 0.8979 - precision: 0.9057 - recall: 0.8875 - f1_score: 0.8963 - val_loss: 2.1147 - val_acc: 0.3676 - val_precision: 0.3723 - val_recall: 0.3382 - val_f1_score: 0.3540
Epoch 41/75
 - 7s - loss: 0.2470 - acc: 0.9187 - precision: 0.9264 - recall: 0.9125 - f1_score: 0.9191 - val_loss: 2.6168 - val_acc: 0.4338 - val_precision: 0.4457 - va